### Command for bulding the localy cloned package

install the package in editable mode (command needs to be copied to the terminal - doens't work in notebook becuase paths or something ¯\_(ツ)_/¯)

""" pip install -e . """

### Example

In [1]:
import torch
from normalizing_flows.flows import Flow, PrincipalManifoldFlow
from normalizing_flows.bijections import RealNVP
# from torch.autograd.functional import vjp

In [ ]:
# Create some synthetic training and validation data
torch.manual_seed(0)

event_shape = (10,)
n_train = 100
n_val = 20
n_test = 10000

x_train = torch.randn(n_train, *event_shape) * 2 + 4
x_val = torch.randn(n_val, *event_shape) * 2 + 4
x_test = torch.randn(n_test, *event_shape) * 2 + 4

x_one_dimensional = torch.randn(event_shape) * 2 + 4

In [ ]:
# Train without validation data
torch.manual_seed(0)
flow0 = PrincipalManifoldFlow(RealNVP(event_shape, n_layers=5), debug=False)
flow0.fit(x_train, show_progress=True)

In [ ]:
# Train with validation data and keep the best weights
torch.manual_seed(0)
flow1 = PrincipalManifoldFlow(RealNVP(event_shape, n_layers=5))
flow1.fit(x_train, show_progress=True, x_val=x_val)

In [ ]:
# Train with validation data, early stopping, and keep the best weights
torch.manual_seed(0)
flow2 = PrincipalManifoldFlow(RealNVP(event_shape, n_layers=5))
flow2.fit(x_train, show_progress=True, x_val=x_val, early_stopping=True)

In [ ]:
print("Test loss values")
print()
print(f"Without validation data: {torch.mean(-flow0.log_prob(x_test))}")
print(f"With validation data, no early stopping: {torch.mean(-flow1.log_prob(x_test))}")
print(f"With validation data, early stopping: {torch.mean(-flow2.log_prob(x_test))}")

### SHAPES EXPERIMENT

In [ ]:
import torch
from normalizing_flows.flows import Flow, PrincipalManifoldFlow
from normalizing_flows.bijections import RealNVP

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, '../')

from test.shapes import *

sys.path.remove('../')

In [ ]:
num_points = 50000

data_regular_polygon = n_regular_polygon(n=10, radius=1, angle=0, arc_height=0.1, num_points=num_points, noise_std=0.02, seed=0)
data_double_circle = double_circle(num_points=num_points, radius1=1.0, radius2=0.5, noise_std=0.02, seed=0)
data_caret = caret(line_length=2, angle=45, num_points=num_points, noise_std=0.05, seed=0)
data_swirl = swirl(radius=1, angle=0, num_points=num_points, noise_std=0.03, seed=0)
data_grid = grid(n=4, dist=(0.5, 0.5), num_points=num_points, noise_std=0.03, seed=0)
data_moons = moons(radius=1, angle=0, shift=None, num_points=num_points, noise_std=0.03, seed=0)
data_swiss_roll = swiss_roll(radius1=1, radius2=0.25, num_revolutions=2, num_points=num_points, noise_std=0.03, seed=0)
data_hline = hline(length=2, num_points=num_points, noise_std=0.03, seed=0)


# Plotting the generated points
ax, fig = plt.subplots(1, 8, figsize=(35, 5))
fig[0].scatter(data_regular_polygon[0], data_regular_polygon[1], s=1)
fig[0].set_title("Regular polygon")
fig[1].scatter(data_double_circle[0], data_double_circle[1], s=1)
fig[1].set_title("Double circle")
fig[2].scatter(data_caret[0], data_caret[1], s=1)
fig[2].set_title("Caret")
fig[3].scatter(data_swirl[0], data_swirl[1], s=1)
fig[3].set_title("Swirl")
fig[4].scatter(data_grid[0], data_grid[1], s=1)
fig[4].set_title("Grid")
fig[5].scatter(data_moons[0], data_moons[1], s=1)
fig[5].set_title("Moons")
fig[6].scatter(data_swiss_roll[0], data_swiss_roll[1], s=1)
fig[6].set_title("Swiss roll")
fig[7].scatter(data_hline[0], data_hline[1], s=1)
fig[7].set_title("Horizontal line")
plt.show()

In [ ]:
# prepare the data
target_data = data_hline

data_tensor = torch.tensor(np.stack(target_data, axis=1)).float()  # Create a tensor from the data
n_dim = 2

In [ ]:
torch.manual_seed(0)
normal_flow = Flow(RealNVP(n_dim, n_layers=10), record_Ihat_P=True, record_log_px=True)
normal_flow.fit(data_tensor, show_progress=True, early_stopping=True)

In [ ]:
torch.manual_seed(0)
PM_flow = PrincipalManifoldFlow(RealNVP(n_dim, n_layers=10), record_Ihat_P=True, record_log_px=True)
PM_flow.fit(data_tensor, show_progress=True)

In [ ]:
x_new_normal = normal_flow.sample(100000).detach().numpy()  # Sample 1000 new data points
x_new_PM = PM_flow.sample(100000).detach().numpy()  # Sample 1000 new data points

# plot new data of both flows (using subplots)
ax, fig = plt.subplots(1, 2, figsize=(10, 5))
fig[0].scatter(x_new_normal[:, 0], x_new_normal[:, 1], s=1)
fig[0].set_title("Normal flow")
fig[0].set_xlim(-1,1)
fig[0].set_ylim(-1,1)
fig[1].scatter(x_new_PM[:, 0], x_new_PM[:, 1], s=1)
fig[1].set_title("PF flow")
fig[1].set_xlim(-1,1)
fig[1].set_ylim(-1,1)

In [ ]:
def gen_grid_lines(x_min=-1, x_max=1, y_min=-1, y_max=1, x_tick=20, y_tick=20, density=100):
    """
    Generate grid points that are densely populated along horizontal and vertical lines at tick intervals,
    with a specified density within each tick line.

    Parameters:
    - x_min, x_max: The minimum and maximum values on the x-axis.
    - y_min, y_max: The minimum and maximum values on the y-axis.
    - x_tick, y_tick: The number of tick lines along the x-axis and y-axis.
    - density: The number of points within each tick line.
    
    Returns:
    - grid: A tensor containing the grid points, densely populated along tick lines.
    """
    # Generate tick positions
    x_ticks = torch.linspace(x_min, x_max, x_tick)
    y_ticks = torch.linspace(y_min, y_max, y_tick)
    
    # Generate dense points along the x-axis for each y-tick
    dense_x = torch.linspace(x_min, x_max, density)
    x_grid = torch.stack([dense_x.repeat(y_tick), torch.tensor([[y]*density for y in y_ticks]).flatten()], dim=-1)
    
    # Generate dense points along the y-axis for each x-tick
    dense_y = torch.linspace(y_min, y_max, density)
    y_grid = torch.stack([torch.tensor([[x]*density for x in x_ticks]).flatten(), dense_y.repeat(x_tick)], dim=-1)
    
    # Combine the grids
    grid = torch.cat((x_grid, y_grid), dim=0).unique(dim=0)
    
    return grid

In [ ]:
def gen_grid_lines_colored(x_min=-1, x_max=1, y_min=-1, y_max=1, x_tick=20, y_tick=20, density=100):
    """
    Generate grid points along horizontal and vertical lines at tick intervals,
    with a specified density within each tick line, and assign colors to each line type.

    Parameters are adjusted for compatibility with the plotting requirements of normalizing flows.
    """
    x_ticks = np.linspace(x_min, x_max, x_tick)
    y_ticks = np.linspace(y_min, y_max, y_tick)
    grid_points = []
    colors = []
    for y in y_ticks:
        for x in np.linspace(x_min, x_max, density):
            grid_points.append([x, y])
            colors.append('blue')  # Horizontal lines
    for x in x_ticks:
        for y in np.linspace(y_min, y_max, density):
            grid_points.append([x, y])
            colors.append('red')  # Vertical lines
    grid = np.array(grid_points)
    return torch.tensor(grid, dtype=torch.float), colors

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Generate grid lines with colors
latent_grid, colors = gen_grid_lines_colored(x_tick=30, y_tick=300, density=3000, x_min=-2, x_max=2, y_min=-2, y_max=2)

# Transform the grid points using the normalizing flows and convert them back to numpy for plotting
latent_grid_PM = PM_flow.bijection(latent_grid)[0].detach().numpy()
latent_grid_normal = normal_flow.bijection(latent_grid)[0].detach().numpy()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

lbound = -2
ubound = 2

# Plot for normal flow
axs[0].scatter(latent_grid_normal[:, 0], latent_grid_normal[:, 1], c=colors, s=1)
axs[0].set_title("Normal flow")
axs[0].set_xlim(lbound, ubound)
axs[0].set_ylim(lbound, ubound)

# Plot for PF flow
axs[1].scatter(latent_grid_PM[:, 0], latent_grid_PM[:, 1], c=colors, s=1)
axs[1].set_title("PF flow")
axs[1].set_xlim(lbound, ubound)
axs[1].set_ylim(lbound, ubound)

plt.show()



Comparison of log_px and Ihat_P values of both flows.

In [ ]:
NF_log_px = normal_flow.get_log_px()
NF_Ihat_P = normal_flow.get_Ihat_P()

# print all
print("NF_log_px: ", len(NF_log_px))
print("NF log_px average: ", torch.mean(NF_log_px))
print("NF_Ihat_P: ", len(NF_Ihat_P))
print("NF Ihat_P average: ", torch.mean(NF_Ihat_P))


In [ ]:
PMF_log_px = PM_flow.get_log_px()
PMF_Ihat_P = PM_flow.get_Ihat_P()

# print all
print("PMF_log_px: ", len(PMF_log_px))
print("PMF log_px average: ", torch.mean(PMF_log_px))
print("PMF_Ihat_P: ", len(PMF_Ihat_P))
print("PMF Ihat_P average: ", torch.mean(PMF_Ihat_P))

In [ ]:
print("PMF last 10 Ihat_P: ", PMF_Ihat_P[-10:])
print("NF last 10 Ihat_P: ", NF_Ihat_P[-10:])

In [ ]:
# plot Ihat_P for both flows on same plot (as different lines)
plt.plot(NF_Ihat_P, label="NF")
plt.plot(PMF_Ihat_P, label="PMF")
plt.legend()
plt.title("Ihat_P")
plt.show()

Comparison of new datapoint generation of both flows.

In [ ]:
x_new_normal = normal_flow.sample(1000).detach().numpy()  # Sample 1000 new data points
x_new_PM = PM_flow.sample(1000).detach().numpy()  # Sample 1000 new data points

# plot new data of both flows (using subplots)
ax, fig = plt.subplots(1, 2, figsize=(10, 5))
fig[0].scatter(x_new_normal[:, 0], x_new_normal[:, 1], s=1)
fig[0].set_title("Normal flow")
fig[1].scatter(x_new_PM[:, 0], x_new_PM[:, 1], s=1)
fig[1].set_title("PF flow")

Comparison of the contours created by both flows.

In [ ]:
x_min, x_max = -1, 1
y_min, y_max = -1, 1
x_tick, y_tick = 20, 20
grid_points = generate_grid_data([x_min, x_max], [y_min, y_max], x_tick, y_tick)

# transform grid points
transformed_grid_points_normal = normal_flow.bijection(grid_points)[0].detach().numpy()
transformed_grid_points_PM = PM_flow.bijection(grid_points)[0].detach().numpy()

# plot grid_points as grid with vertical lines being black and horizontal lines being red
ax, fig = plt.subplots(1, 3, figsize=(15, 5))
for i in range(x_tick):
    fig[0].axvline(x=grid_points[i*y_tick,0], color='black')
    fig[1].plot(transformed_grid_points_normal[i*y_tick:i*y_tick+y_tick,0], transformed_grid_points_normal[i*y_tick:i*y_tick+y_tick,1], color='black')
    fig[2].plot(transformed_grid_points_PM[i*y_tick:i*y_tick+y_tick,0], transformed_grid_points_PM[i*y_tick:i*y_tick+y_tick,1], color='black')
for i in range(y_tick):
    fig[0].axhline(y=grid_points[i,1], color='red')
    fig[1].plot(transformed_grid_points_normal[i::y_tick,0], transformed_grid_points_normal[i::y_tick,1], color='red')
    fig[2].plot(transformed_grid_points_PM[i::y_tick,0], transformed_grid_points_PM[i::y_tick,1], color='red')

# add titles
fig[0].set_title("Grid in latent space")
fig[1].set_title("Normal flow")
fig[2].set_title("PF flow")

plt.show()


### Example case for understanding dimensions

In [ ]:
# create 1 d tensor data
z = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32)

print(z.shape)
print(z.shape[-1])

# create 2 d tensor data
zz = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], dtype=torch.float32)

print(zz.shape)
print(zz.shape[-1])